<a href="https://colab.research.google.com/github/gulayoklan/Ceng463-Assignment2/blob/main/inference_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Read and process data for inference.

In [ ]:
file_path = "orientation-it-train.tsv"

# Load the dataset
df = pd.read_csv(file_path, sep='\t')

# Keep only the necessary columns (text and label) and drop rows with missing values
df = df[['text_en', 'text' ,'label']].dropna()

# Ensure the label column is binary and of type integer
df['label'] = df['label'].astype(int)


In [ ]:
from huggingface_hub import login

login() # It will prompt you to enter your token

In [ ]:
from transformers import pipeline
model_name = "meta-llama/Llama-3.1-8B"
generator = pipeline(
    "text-generation",
    model=model_name,
    device=-1,    # model at cpu
)

In [ ]:
def classify_with_generation(
    generator_pipeline,
    text,
    candidate_labels,
    max_new_tokens=32,
    do_sample=True,
    repetition_penalty=1.0
):
    labels_str = ", ".join(candidate_labels)
    #provide the prompt appropriate to task.
    prompt2=(
        "You are a highly intelligent political analyst and language model trained to classify parliamentary texts based on the speaker's political orientation.\n "
        "Your task is to determine whether the speaker's orientation is 'right' or 'left' based on the provided parliamentary speech. Respond with only the label ('right' or 'left') and nothing else.\n"
        f"Here is the parliamentary text:{text}\n"
        "Label:"
      )

    # 2) Generate output from the model
    output = generator_pipeline(
        prompt2,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=0.2,
        repetition_penalty=repetition_penalty
    )

    # The pipeline returns a list of dicts, e.g. [{'generated_text': "..."}].
    generated_text = output[0]["generated_text"]


    best_label = None
    # Convert to lowercase for simpler matching (optional).
    gen_lower = generated_text.lower()

    for label in candidate_labels:
        if label.lower() in gen_lower[-17:]:
            best_label = label
            break

    return best_label, generated_text

def llm_inference(texts):
  candidate_labels = ["left", "right"]
  predictions = []
  true_labels = []
  labels = test.label.values

  wrong_label=0
  for i in range(0,len(texts)):

    text = texts[i]
    label = labels[i]
    best_label, raw_output = classify_with_generation(
        generator_pipeline=generator,
        text=text,
        candidate_labels=candidate_labels,
        max_new_tokens=3,
        do_sample=False,
        repetition_penalty=1.2
    )
    if (best_label=="left"):
      predictions.append(0)
      true_labels.append(label)
    elif (best_label=="right"):
      predictions.append(1)
      true_labels.append(label)
    else:
      wrong_label+=1
  return predictions,true_labels,wrong_label



In [ ]:
it_predictions,it_true_labels,it_wrong_label=llm_inference(df.text.values)
predictions,true_labels,wrong_label=llm_inference(df.text_en.values)


In [ ]:
print(classification_report(it_true_labels,it_predictions))
print(it_wrong_label)
print(np.unique(it_predictions, return_counts=True))

In [ ]:
print(classification_report(true_labels,predictions))
print(wrong_label)
print(np.unique(predictions, return_counts=True))